In [2]:
import sys
from collections import deque
from string import strip, replace
import json
from collections import Counter
sys.path.append("../../../parsetrees/expr_parser/")


In [4]:
cd "../../../parsetrees/expr_parser/"

/Users/maningmn1987/Documents/Study_material/CSE103_TA/Webwork_AdaptiveHints/src/parsetrees/expr_parser


In [5]:
!ls 

10.18.13.prob3.json                   example_struggle.csv
AnswersByPartFromMySQL.ipynb          example_struggle.json
BrowseAnswersInteractive.ipynb        extract_answers.py
ComputeEditDistances-Yoav.ipynb       k_medoids_clustering.py
ComputeEditDistances.ipynb            parser.out
Eval_parsed.py                        parsetab.py
Eval_parsed.pyc                       parsetab.pyc
EvaluateExpressions.ipynb             poker_cond2_1.pg.json
GetAnswersByPart.ipynb                test
GetAnswersByPartAll.ipynb             webwork_ipython_utils.py
LoadingParsingEditDistance.ipynb      webwork_lexer.py
Parsing Answers from API.ipynb        webwork_lexer.pyc
Study Parsing And Edit Distance.ipynb webwork_parser.py
Webwork XMLRPC.ipynb                  webwork_parser.py.bck
__init__.py                           webwork_parser.pyc


In [6]:
cd "../../servers/rest_server/filter_lab"

/Users/maningmn1987/Documents/Study_material/CSE103_TA/Webwork_AdaptiveHints/src/servers/rest_server/filter_lab


In [7]:
from Eval_parsed import eval_parsed
from webwork_parser import parse_webwork

# Set up a logging object
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)
#logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)




In [91]:
def flatten(tree,tag):
    #attempt_parsed_eval [['/', 0.451171875, [0, 7], 'R'], ['X', 462.0, [0, 2], 'R.0'], [['^', 1024.0, [4, 7], 'R.1'], ['X', 2.0, [4, 4], 'R.1.0'], ['X', 10.0, [6, 7], 'R.1.1']]]
    logger.debug('flattening: '+str(tree))
    #print 'the tree to be flatten is ', tree
    List=[]
    Queue=deque([tree])
    try:
        while Queue:
            current=Queue.popleft()
            #print "current is ", current
            logger.debug('current='+str(current)+' Queue='+str(Queue))
            if type(current)==list: 
                if type(current[0])==str:
                    List.append([current[1],tag,current])
                    #print "currt list", List
                elif isinstance(current[0][0],basestring) and isinstance(current[0][1],(int, long, float, complex)) and type(current[0][2])==list:
                    List.append([current[0][1],tag,current])
                    #print "currt list", List
                    Queue.extend(current[1:])
                else:
                    logger.error('parse error (is list): current=%s, tree=%s'%(str(current),str(tree)))
                    return []
            else:
                logger.error('parse error (not list): current=%s, tree=%s'%(str(current),str(tree)))
                return []
    except Exception as error:
        logger.exception(error)
        logger.error('current='+str(current))
        return []
    List=sorted(List,key=lambda x: x[0])
    #print 'List after sort', List
    return List



In [92]:
def find_Hits(List,tol = 1+1e-6):
    """ Given a combined list of subtrees from both attempt and answer,
    sorted by value, find the matching pairs of trees
    tol is the tolerance used to define which pairs of values match. 
    Needed because different expressions get different roundoff error
    """
    Hits=[]
    item1=List[0]
    for item2 in List[1:]:
        logger.debug(str(item1[:2])+str(item2[:2]))
        ratio=item1[0]/item2[0]
        if item1[1]!=item2[1] and ratio <tol and ratio>(1/tol):
            if item1[1]=='c':
                Hits.append((item1,item2))
            elif item2[1]=='c':
                Hits.append((item2,item1))
            else:
                logger.error("Error in find_Hits. Neither item labeled c")
        item1=item2
    return Hits



In [93]:
#return the location of the evaluation
def get_span(tree):
    if tree[2][0]=='X':
        return tree[2][2]
    elif type(tree[2][0])==list:
        return tree[2][0][2]
    else:
        logger.error('Error in get_span')
        return None
    


In [106]:
#each element in hints is
#([2.0, 'c', ['X', 2.0, [18, 18], 'R.1.0']], [2.0, 't', ['X', 2.0, [4, 4], 'R.1.0']])

#([462.0, 'c', [['/', 462.0, [1, 15], 'R.0'], [['!', 39916800, [1, 3], 'R.0.0'], ['X', 11.0, [1, 2], 'R.0.0.0']], [['*', 86400.0, [6, 14], 'R.0.1'], [['!', 720, [6, 7], 'R.0.1.0'], ['X', 6.0, [6, 6], 'R.0.1.0.0']], [['!', 120, [8, 14], 'R.0.1.1'], [['-', 5.0, [9, 12], 'R.0.1.1.0'], ['X', 11.0, [9, 10], 'R.0.1.1.0.0'], ['X', 6.0, [12, 12], 'R.0.1.1.0.1']]]]]], 
#[462.0, 't', ['X', 462.0, [0, 2], 'R.0']])

In [124]:

def find_dominating_hits(Hits,answer,attempt):
    for i in range(len(Hits)):
        for j in range(len(Hits)):
            if i==j:
                continue
            print 'i, j', i, j
            print 'Hists[i][0]', Hits[i][0]
            span1=get_span(Hits[i][0])
            print 'span1', span1
            print 'Hists[j][0]', Hits[j][0]
            span2=get_span(Hits[j][0])
            print 'span2', span2
            if span1[0]<=span2[0] and span1[1]>=span2[1]:
                logger.debug(str(span1)+' dominates '+str(span2))
                Hits[j][0][1]='dc'
    print
    print 'each element after dominiate Hint'
    for ele in Hits:
        print ele;
        print
    print
    
    final_matches=[]
    for hit in Hits:
        if hit[0][1]=='c':
            value=hit[0][0]
            span_c=get_span(hit[0])
            span_a=get_span(hit[1])
            logger.debug('answer:'+str(answer)+str(span_c))
            logger.debug('attempt:'+str(attempt)+str(span_a))
            answer_part=answer[span_c[0]:span_c[1]+1]
            attempt_part = attempt[span_a[0]:span_a[1]+1]
            logger.debug('answer part'+str(answer_part))
            logger.debug('attempt part'+str(attempt_part))
            logger.debug('hit part='+str(hit[0][2][0]))
            if hit[0][2][0]=='X':
                node=hit[0][2][3]
            else:
                node=hit[0][2][0][3]
            final_matches.append((node,value,answer_part,attempt_part))
    return final_matches

In [125]:
def find_matches(answer,attempt):

    logger.debug('find_matches recieved answer='+str(answer)+' attempt='+str(attempt))
    attempt_parsed=parse_webwork(attempt)
    print 'attempt_parsed', attempt_parsed
    print
    logger.debug('calling eval_parsed on on'+str(attempt_parsed))
    attempt_tree=eval_parsed(attempt_parsed)
    print 'attempt_parsed_eval', attempt_tree
    logger.debug('calling flatten on'+str(attempt_tree))
    attempt_list=flatten(attempt_tree,'t')
    print
    print 'each element in attempt_parsed_eval is '
    for ele in attempt_tree:
        print ele
    print
    print 'attempt_parsed_eval_flat', attempt_list
    logger.debug( 'attempt list:\n'+str(attempt_list))
    print
    print 'each element in attempt_parsed_eval_flat is '
    for ele in attempt_list:
        print ele
    print
    #print 'attempt_parsed_eval_flat', attempt_list
    print 'the value in attempt_parsed_eval_flat is '
    for ele in attempt_list:
        print ele[0]
    
    print
    answer_parsed=parse_webwork(answer)
    print 'answer_parsed', answer_parsed
    logger.debug('calling eval_parsed on on'+str(answer_parsed))
    answer_tree=eval_parsed(answer_parsed)
    print 'answer_parsed_eval', answer_tree
    print
    print 'each element in answer_parsed_eval is '
    for ele in answer_tree:
        print ele
    print
    logger.debug('calling flatten on'+str(answer_tree))
    answer_list=flatten(answer_tree,'c')
    print 'answer_parsed_eval_flat', answer_list
    print
    print 'each element in answer_parsed_eval_flat is '
    for ele in answer_list:
        print ele
    print
    logger.debug('answer list\n'+str(answer_list))
    print 'the value in answer_parsed_eval_flat is '
    for ele in answer_list:
        print ele[0]
    
    print
    combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])
    print 'combined_list', combined_list
    logger.debug('combined list\n'+str(combined_list))
    
    print
    Hits=find_Hits(combined_list)
    print 'Hitsssssss:', Hits
    logger.debug('Hits:\n'+str(Hits))
    print
    print "each element in hints is"
    for ele in Hits:
        print ele
        print
    
    print
    final_matches=find_dominating_hits(Hits,answer,attempt)
    print 'final_matches:', final_matches
    
    return final_matches

In [151]:
file=open('CSE103_Fall14_Week2_problem13_part1.txt','r')
print file.readline(),
answer_expression=strip(file.readline())
print "|%s|"%answer_expression
Clusters=Counter()
Reps={}

k = 0
for line in file.readlines():
    if k > 1:
        break
    k = k + 1
    #attempt,att_parse_tree,att_eval_tree,answer,answer_parse_tree,answer_eval_tree,variables = params
    params=json.loads(line)
    print 'params: ', params;
    attempt=params[0]
    print 'parans[0]: ', attempt
    attempt = ''.join(attempt.split())
    print 'ATTEMPT (AFTER JOIN): ', attempt
    variables=params[6]
    print 'variables: ', variables
    answer=answer_expression
    print 'Answer: ', answer_expression
   
    
    for name,val in variables.items():
        print 'name:', name, ' val:', val
        if int(val)==val:
            val=int(val)
        answer=answer.replace(name,str(val))
        print 'ANSWER (after replace)', answer
    print
    
    print 'attempt=',attempt,'answer=',answer
    print "find_matches from .json"
    final_pairs=find_matches(answer,attempt)
    #print "find_matches from arguement"
    #final_paris_argument = find_matches('(11!/(6!(11-6)!))/2^11','462/2^10')
    #print final_paris_argument
    if len(final_pairs)>0:
        for node,value,ans_piece,attempt_piece in final_pairs:
            if value>10 or value != int(value):
                print '-'*50
                print 'attempt=',attempt,'answer=',answer
                if node=='R':
                    sub_type='answer'
                else:
                    sub_type='sub-expression'
                if not node in Reps.keys():
                    Reps[node]=ans_piece
                Clusters[node]+=1
                if attempt_piece != ans_piece:
                    print 'The %s %s is correct, it could also be written as %s'%(sub_type,attempt_piece,ans_piece)
                else:
                    print 'The %s %s is correct'%(sub_type,attempt_piece)
            else:
                Clusters['Nothing']+=1
        
print 'Clusters='
print 'Nothing recognizable=',Clusters['Nothing']
for node in Reps.keys():
    print "%20s%30s%10d"%(node,Reps[node],Clusters[node])

Week2 Problem13 Part1
|2^$coins|
params:  [u'2', 2, [2.0], u'2^$coins', [[u'^', [0, 2]], 2, u'$coins'], [[u'^', 2048.0, [0, 2]], [2.0], [11.0]], {u'$coins': 11.0}]
parans[0]:  2
ATTEMPT (AFTER JOIN):  2
variables:  {u'$coins': 11.0}
Answer:  2^$coins
name: $coins  val: 11.0
ANSWER (after replace) 2^11

attempt= 2 answer= 2^11
find_matches from .json
attempt_parsed [['X', [0, 0]], 2]

attempt_parsed_eval ['X', 2.0, [0, 0], 'R']

each element in attempt_parsed_eval is 
X
2.0
[0, 0]
R

attempt_parsed_eval_flat [[2.0, 't', ['X', 2.0, [0, 0], 'R']]]

each element in attempt_parsed_eval_flat is 
[2.0, 't', ['X', 2.0, [0, 0], 'R']]

the value in attempt_parsed_eval_flat is 
2.0

answer_parsed [['^', [0, 3]], [['X', [0, 0]], 2], [['X', [2, 3]], 11]]
answer_parsed_eval [['^', 2048.0, [0, 3], 'R'], ['X', 2.0, [0, 0], 'R.0'], ['X', 11.0, [2, 3], 'R.1']]

each element in answer_parsed_eval is 
['^', 2048.0, [0, 3], 'R']
['X', 2.0, [0, 0], 'R.0']
['X', 11.0, [2, 3], 'R.1']

answer_parsed_eval_flat 

In [137]:
attempt

u'462/2^10'

In [99]:
a = ['s', 'b', 'c']


In [30]:
a

['s', 'b', 'c']

In [55]:
':'.join(a)

's:b:c'

In [64]:
myList = [[1,2], [3,5], ['X',2], [4.0,3]]

In [65]:
  myList=sorted(myList,key=lambda x: x[0])

In [66]:
myList

[[1, 2], [3, 5], [4.0, 3], ['X', 2]]